In [1]:
#1.加载数据
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

pages = loader.load_and_split()

In [1]:
from langchain.document_loaders import TextLoader

# 使用 TextLoader 来加载 TXT 文件
loader = TextLoader("初中九年级化学实验.txt")

# 加载和分割文本内容
pages = loader.load_and_split()

In [2]:
pages[0]

Document(metadata={'source': '初中九年级化学实验.txt'}, page_content='【水的沸腾】：准备一个烧杯，加入适量的水。将烧杯放在酒精灯或电热板上加热。观察水的温度变化，并用温度计记录数据。观察水开始冒泡的现象，记录水沸腾的温度（100°C）。\n\n【胆矾的研碎】：取适量胆矾（硫酸铜晶体）放入研钵中。使用研杵轻轻按压和研磨胆矾，直到其变成粉末状态为止。观察研碎后的胆矾外观，记录其颜色和质地的变化。\n\n【往硫酸铜溶液中滴加氢氧化钠溶液】：准备一个试管，加入适量的硫酸铜溶液（蓝色）。使用滴管，慢慢向试管中滴加氢氧化钠溶液。观察反应现象，记录氢氧化钠溶液加入后的颜色变化（出现蓝色沉淀）。\n\n【石灰石与盐酸的反应】：将适量的石灰石（碳酸钙）放入一个烧杯或试管中。向烧杯中滴加适量的盐酸（稀盐酸）。观察反应，记录气泡产生的情况（二氧化碳气体的生成）。可以用气体产生的泡沫去检验不再产生泡沫时的反应完成度。\n\n【氧气和二氧化碳的有关性质】：准备一个装有氧气的试管和一个装有二氧化碳的试管。取一根易燃物（如火柴或蜡烛），将其点燃。将点燃的易燃物放入装有氧气的试管中，观察燃烧现象（燃烧加剧）。将点燃的易燃物放入装有二氧化碳的试管中，观察现象（火焰熄灭）。记录观察到的氧气和二氧化碳的性质以及它们对燃烧的影响。\n\n【固体药品的取用】 : 首先，准备好所需的固体药品和药匙，确保药品在清洁的实验环境中使用。然后，打开药品的容器，用干净的药匙轻轻搅拌药品，确保其均匀。接着，用药匙小心地取出适量药品，观察药品的性质（如颜色、颗粒大小等），最后将取用的药品放入干净的容器中，避免交叉污染，并将未用完的药品按照实验室规定妥善保存。\n\n【在澄清石灰水中滴加酚酞溶液】 : 先准备一个装有澄清石灰水的试管，然后用滴管吸取适量的酚酞溶液。接着，将酚酞溶液缓慢滴加到石灰水中，观察溶液颜色的变化。当酚酞溶液与石灰水反应后，如果石灰水变为桃红色，说明其为碱性溶液。\n\n【在大理石或碳酸钠粉末中加入稀盐酸】 : 取适量的大理石或碳酸钠粉末放入一个试管中，随后用量筒或滴管吸取适量稀盐酸。将稀盐酸缓慢倒入试管中，观察反应现象，注意观察是否有气泡产生，记录反应的剧烈程度及气泡数量，反应完毕后，确保安全处理试管中的物质。\n\n【观察酒精灯火焰；将火柴梗平放如火

In [14]:
with open("初中九年级化学实验.txt", 'r', encoding='utf-8') as file:
    lines = file.readlines()  # 读取所有行

line_lengths = [len(line.strip()) for line in lines]  # 去掉每行首尾的空白字符，计算长度
average_length = sum(line_lengths) / len(line_lengths)  # 计算平均长度
average_length

47.324561403508774

In [18]:
import statistics
statistics.median(line_lengths)

0.0

In [45]:
#2.知识切片 将文档分割成均匀的块。每个块是一段原始文本
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 140,
    chunk_overlap = 14,
)

docs = text_splitter.split_documents(pages)

In [46]:
len(docs)

114

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os


embed_model = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base="https://api.chatanywhere.tech/v1"
)


In [47]:
vectorstore1 = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed2",persist_directory="task_data")

In [24]:
query =   """操作1：检查装置气密性
        用火柴1点燃酒精灯，观察导管口是否会有连续、均匀气泡冒出 。

        操作2：熄灭酒精灯进行正式制取实验过程
        将酒精灯帽盖在酒精灯上。

        操作3：取药品
        打开装有高猛酸钾的塞，用药匙取高猛酸钾

        操作4：装药品
        将高锰酸钾药品平铺在试管底部，管口放一小团棉花，用带导管的单孔橡皮塞塞紧试管口。并固定在铁架台上

        操作5：点燃酒精灯加热
        用火柴2点燃酒精灯

        操作6：观察导管口是否会有连续、均匀气泡冒出，利用集气瓶收集气体
        操作7：实验结束
        """

In [50]:
vectorstore1.similarity_search(query ,k = 3)

[Document(metadata={'source': '初中九年级化学实验.txt'}, page_content='首先，准备一个干燥的试管，加入适量的高锰酸钾，并将试管固定在试管夹中。然后，将试管放置在一个加热器上，缓慢加热高锰酸钾，观察反应过程中是否有气体产生。使用水槽收集气体，若有气泡生成，则说明在加热过程中产生了氧气。此过程需要持续加热，直到不再有气泡出现，最后安全地处理反应后的残留'),
 Document(metadata={'source': '初中九年级化学实验.txt'}, page_content='【观察酒精灯火焰；将火柴梗平放如火焰中】 : 首先，准备一个酒精灯，打开灯芯并点燃，观察火焰的形状和颜色。接着，取一根火柴梗，将其平放在火焰中，观察火柴梗的燃烧情况，记录火焰的状态以及火柴梗燃烧的效果，如火焰的温度和颜色变化。'),
 Document(metadata={'source': '初中九年级化学实验.txt'}, page_content='【分解过氧化氢溶液制取氧气】 : 在试管中加入过氧化氢，并加入二氧化锰作为催化剂，观察气体生成并记录气泡的数量和速度，分析反应特点及生成气体的性质。\n\n【加热氯酸钾制取氧气】 : 准备氯酸钾放入耐热容器中，加热至其分解，观察气体的产生，记录反应现象，分析获得的氧气的性质。')]

In [ ]:
vectorstore_test = Chroma(persist_directory="task_data", embedding_function=embed_model, collection_name="openai_embed")

C:\Users\ye_kkk\AppData\Local\Temp\ipykernel_43808\3760694070.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_test = Chroma(persist_directory="task_data", embedding_function=embed_model, collection_name="openai_embed")


In [13]:
vectorstore_test.similarity_search(query ,k = 2)

[Document(metadata={'source': '初中九年级化学实验.txt'}, page_content='【水的沸腾】：准备一个烧杯，加入适量的水。将烧杯放在酒精灯或电热板上加热。观察水的温度变化，并用温度计记录数据。观察水开始冒泡的现象，记录水沸腾的温度（100°C）。\n\n【胆矾的研碎】：取适量胆矾（硫酸铜晶体）放入研钵中。使用研杵轻轻按压和研磨胆矾，直到其变成粉末状态为止。观察研碎后的胆矾外观，记录其颜色和质地的变化。\n\n【往硫酸铜溶液中滴加氢氧化钠溶液】：准备一个试管，加入适量的硫酸铜溶液（蓝色）。使用滴管，慢慢向试管中滴加氢氧化钠溶液。观察反应现象，记录氢氧化钠溶液加入后的颜色变化（出现蓝色沉淀）。\n\n【石灰石与盐酸的反应】：将适量的石灰石（碳酸钙）放入一个烧杯或试管中。向烧杯中滴加适量的盐酸（稀盐酸）。观察反应，记录气泡产生的情况（二氧化碳气体的生成）。可以用气体产生的泡沫去检验不再产生泡沫时的反应完成度。'),
 Document(metadata={'source': '初中九年级化学实验.txt'}, page_content='【鲜花变色】 : 准备白色鲜花、清水和食用色素。将食用色素加入水中，待其溶解后，将花茎切口插入溶液中，观察鲜花的颜色变化，记录不同颜色的吸收效果，分析花朵吸水的情况。\n\n【制作“叶脉书签”】 : 准备一些树叶，使用水以去除叶肉。将去除肉的叶脉放入清水中，等待几天晾干后。记录叶脉的完整状态，制作书签，并观察和讨论它的美观与用途。\n\n【有关保鲜膜的实验】 : 准备一些食物和不同类型的保鲜膜。记录每种保鲜膜包裹食物后的保鲜效果（如水分流失、变质程度等），比较保鲜膜的优劣，分析其原理。\n\n【液体药品的取用】 : 准备量筒、气囊和液体药品。用量筒量取所需液体，用气囊无污染地转移到另一个容器中，观察和记录寒温度影响下药品的性质变化。\n\n【酒精灯的使用方法】 : 先检查酒精灯的燃料是否充足，点燃酒精灯，观察灯火焰的形状和颜色。确保其放置在安全的位置，记录火焰的高低、温度等，学习如何安全使用酒精灯。\n\n【托盘天平的使用方法】 : 准备一个托盘天平和一些标准物质。依次将标准平方米放在天平上，学习如何进行称量及其调整过程，记录测量的准确性及误差来源。')]

In [17]:
#没有openaikey
#1.embeding模型
#2.chat模型
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "sentence-transformers/sentence-t5-large"


In [18]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

In [19]:
vectorstore_hf1 = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed",persist_directory="chroma_data1")

In [20]:
resultn = vectorstore_hf1.similarity_search(query ,k = 2)
resultn

[Document(page_content='【乙醇能溶解在水中】 : 首先，准备一个量杯，倒入适量的水。然后，取适量乙醇，缓慢加入水中。观察乙醇与水混合的情况，记录混合后的颜色和清澈度变化，以判断乙醇的溶解性，并分析其物理性质。\n\n【乳浊液的形成和乳化现象】 : 首先，准备一个透明的容器，加入适量的水，并向其中添加一些油。然后，用搅拌器或搅拌棒来搅拌混合物，观察乳浊液的形成。记录乳浊液的颜色和粘度变化，分析乳化现象的性质以及其形成的条件。\n\n【氯化钠在水中的溶解】 : 首先，准备一个透明的量杯，倒入适量的水。然后，取适量的氯化钠，缓慢地加入水中，并用搅拌棒轻轻搅拌。观察氯化钠在水中的溶解情况，记录溶解速率和完成后的溶液透明度，以分析氯化钠的溶解性质。\n\n【硝酸钾在水中的溶解】 : 首先，准备一个透明的量杯，加入适量的水。然后，取适量硝酸钾，缓慢地加入水中，同时用搅拌棒轻轻搅拌。观察硝酸钾的溶解过程，记录并分析完成后的溶液透明度及溶解速率。', metadata={'source': '初中九年级化学实验.txt'}),
 Document(page_content='【向浑浊的水中加入明矾粉末】 : 首先，取一杯浑浊的水，并准备适量的明矾粉末。然后，缓慢地将明矾粉末加入浑浊水中，轻轻搅拌使其充分溶解。观察混合后水的变化，记录明矾与水的反应过程以及水的澄清程度，注意反应过程中的沉淀情况。\n\n【过滤】 : 首先，准备好一套过滤装置，包括漏斗和滤纸。将滤纸放入漏斗中，并将漏斗置于干净的收集容器上。然后，将加入明矾前的浑浊水缓慢倒入滤纸中，观察过滤后的水和滤纸上的沉淀物，记录过滤的效率和结果，分析水的净化效果。\n\n【用肥皂水区分软水和硬水】 : 首先，准备两杯水，一杯为软水，一杯为硬水。然后，向每杯水中分别加入适量的肥皂水，并用搅拌棒轻轻搅拌。观察水中的泡沫情况，记录软水与硬水的泡沫产生情况和稳定性，分析肥皂在软水和硬水中的效果差异。\n\n【实验室制取蒸馏水】 : 首先，准备好蒸馏装置，包括圆底烧瓶和冷凝管。将待蒸馏的水或混合溶液倒入圆底烧瓶中，并确保连接部分气密。然后，用加热器加热烧瓶中的液体，观察蒸气的产生并随其凝结成液体流入接收容器中，记录蒸馏水的收集量和质量。', metadata={'source': '初中九年级化学实验.txt'})]

In [23]:
vectorstore_hf = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed",persist_directory="chroma_data")

In [5]:
query = "How large is the baichuan2 vocabulary?"

In [25]:
result = vectorstore_hf.similarity_search(query ,k = 2)

In [26]:
print(result)

[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': 'C:\\Users\\ye_kkk\\AppData\\Local\\Temp\\tmp_2u6nnkm\\tmp.pdf'}), Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baich

In [6]:
vectorstore = Chroma(persist_directory="chroma_data", embedding_function=embedding, collection_name="huggingface_embed")


In [34]:
vectorstore

In [31]:
vectorstore_hf

In [7]:
result1 = vectorstore.similarity_search(query ,k = 2)

In [8]:
print(result1)

[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': 'C:\\Users\\ye_kkk\\AppData\\Local\\Temp\\tmp_2u6nnkm\\tmp.pdf'}), Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baich